## Use R as control plane language to kick off an R job in Azure Machine Learning

This has some shortcomings, so I would recommend using Python to kick off your R jobs instead of R. But, if you really don't want to touch Python, then here are the required steps (using reticulate to call from R into Python).

### Load reticulate and set the right python (the one that has the azureml sdk installed)

In [1]:
library(reticulate)
use_python("/home/nbuser/anaconda3_501/bin/python")

In [2]:
azureml_core = import("azureml.core")

### Load Workspace
The following cell will read the workspace configuration from aml_config/config.json

Make sure to place the config.json file containing your workspace configuration there -- see <a href="https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py#from-config-path-none--auth-none-">here</a> for details. 

In [3]:
ws = azureml_core$Workspace$from_config()

### Create (or get) an experiment

In [6]:
experiment = azureml_core$Experiment(ws, 'test-R')

### Fetch a compute target from my workspace
This is a DSVM, but AML Compute is fine, too

In [47]:
# list all compute targets
ws$compute_targets

$adftest
<azureml.core.compute.datafactory.DataFactoryCompute>

$lisdvm
<azureml.core.compute.remote.RemoteCompute>

$p100cluster
<azureml.core.compute.amlcompute.AmlCompute>


In [31]:
compute_target = ws$compute_targets$lisdvm

### Submit a run with the file test.R in this folder

Since R is not in the default environment created by Azure ML, im adding the basic R conda packages `'r-essentials'` and `'mro-base'`. I am also addeing `'r-reticulate'` to be able to call the AML apis from the run in order to log metrics during the run.

In [32]:
azureml_train_estimator = import("azureml.train.estimator")
Estimator = azureml_train_estimator$Estimator

estwithr = Estimator(source_directory='.', 
                     conda_packages=c('r-essentials', 'mro-base', 'r-reticulate'),
                     compute_target=compute_target, 
                     entry_script='launchR.py', 
                     script_params=dict('test.R'=''))

In [36]:
runwithr = experiment$submit(estwithr)

### Wait for the job to complete 
Unfortunately, show_output=TRUE does not stream the output to the notebook. Neither does the Widget work -- so you will need to go to http://portal.azure.com to see the progress of your run.

In [40]:
runwithr$wait_for_completion(show_output=TRUE)

$runId
[1] "test-R_1544592066667"

$target
[1] "lisdvm"

$status
[1] "Completed"

$startTimeUtc
[1] "2018-12-12T05:21:10.269702Z"

$endTimeUtc
[1] "2018-12-12T05:21:20.606379Z"

$properties
$properties$azureml.runsource
[1] "experiment"

$properties$ContentSnapshotId
[1] "ce1a5c01-cd88-46b0-9122-67b77b24768b"


$runDefinition
$runDefinition$Script
[1] "launchR.py"

$runDefinition$Arguments
[1] "test.R"

$runDefinition$SourceDirectoryDataStore
NULL

$runDefinition$Framework
[1] 0

$runDefinition$Communicator
[1] 0

$runDefinition$Target
[1] "lisdvm"

$runDefinition$DataReferences
named list()

$runDefinition$JobName
NULL

$runDefinition$AutoPrepareEnvironment
[1] TRUE

$runDefinition$MaxRunDurationSeconds
NULL

$runDefinition$NodeCount
[1] 1

$runDefinition$Environment
$runDefinition$Environment$Python
$runDefinition$Environment$Python$InterpreterPath
[1] "python"

$runDefinition$Environment$Python$UserManagedDependencies
[1] FALSE

$runDefinition$Environment$Python$CondaDependencies
$runDefinition$Environment$Python$CondaDependencies$name
[1] "project_environment"

$runDefinition$Environment$Python$CondaDependencies$dependencies
$runDefinition$Environment$Python$CondaDependencies$dependencies[[1]]
[1] "python=3.6.2"

$runDefinition$Environment$Python$CondaDependencies$dependencies[[2]]
$runDefinition$Environment$Python$CondaDependencies$dependencies[[2]]$pip
[1] "azureml-defaults"


$runDefinition$Environment$Python$CondaDependencies$dependencies[[3]]
[1] "r-essentials"

$runDefinition$Environment$Python$CondaDependencies$dependencies[[4]]
[1] "mro-base"

$runDefinition$Environment$Python$CondaDependencies$dependencies[[5]]
[1] "r-reticulate"



$runDefinition$Environment$Python$CondaDependenciesFile
NULL


$runDefinition$Environment$EnvironmentVariables
$runDefinition$Environment$EnvironmentVariables$EXAMPLE_ENV_VAR
[1] "EXAMPLE_VALUE"


$runDefinition$Environment$Docker
$runDefinition$Environment$Docker$BaseImage
[1] "mcr.microsoft.com/azureml/base:0.2.0"

$runDefinition$Environment$Docker$Enabled
[1] TRUE

$runDefinition$Environment$Docker$SharedVolumes
[1] TRUE

$runDefinition$Environment$Docker$Preparation
NULL

$runDefinition$Environment$Docker$GpuSupport
[1] FALSE

$runDefinition$Environment$Docker$Arguments
list()

$runDefinition$Environment$Docker$BaseImageRegistry
$runDefinition$Environment$Docker$BaseImageRegistry$Address
NULL

$runDefinition$Environment$Docker$BaseImageRegistry$Username
NULL

$runDefinition$Environment$Docker$BaseImageRegistry$Password
NULL



$runDefinition$Environment$Spark
$runDefinition$Environment$Spark$Repositories
[1] "https://mmlspark.azureedge.net/maven"

$runDefinition$Environment$Spark$Packages
$runDefinition$Environment$Spark$Packages[[1]]
$runDefinition$Environment$Spark$Packages[[1]]$Group
[1] "com.microsoft.ml.spark"

$runDefinition$Environment$Spark$Packages[[1]]$Artifact
[1] "mmlspark_2.11"

$runDefinition$Environment$Spark$Packages[[1]]$Version
[1] "0.12"



$runDefinition$Environment$Spark$PrecachePackages
[1] TRUE



$runDefinition$History
$runDefinition$History$OutputCollection
[1] TRUE


$runDefinition$Spark
$runDefinition$Spark$Configuration
$runDefinition$Spark$Configuration$spark.app.name
[1] "Azure ML Experiment"

$runDefinition$Spark$Configuration$spark.yarn.maxAppAttempts
[1] "1"



$runDefinition$BatchAi
$runDefinition$BatchAi$NodeCount
[1] 0


$runDefinition$AmlCompute
$runDefinition$AmlCompute$Name
NULL

$runDefinition$AmlCompute$VmSize
NULL

$runDefinition$AmlCompute$VmPriority
NULL

$runDefinition$AmlCompute$RetainCluster
[1] FALSE

$runDefinition$AmlCompute$ClusterMaxNodeCount
[1] 1


$runDefinition$Tensorflow
$runDefinition$Tensorflow$WorkerCount
[1] 1

$runDefinition$Tensorflow$ParameterServerCount
[1] 1


$runDefinition$Mpi
$runDefinition$Mpi$ProcessCountPerNode
[1] 1


$runDefinition$Hdi
$runDefinition$Hdi$YarnDeployMode
[1] 2


$runDefinition$ContainerInstance
$runDefinition$ContainerInstance$Region
NULL

$runDefinition$ContainerInstance$CpuCores
[1] 0

$runDe

In [42]:
runwithr$

$id
[1] "/subscriptions/a947bb9f-f570-40a9-82cc-2fdd09f1553a/resourceGroups/AMLworkshop/providers/Microsoft.MachineLearningServices/workspaces/DanielSc"

$name
[1] "DanielSc"

$location
[1] "westeurope"

$type
[1] "Microsoft.MachineLearningServices/workspaces"

$workspaceid
[1] "522ab8ca-c5a7-484b-b47e-f197ccec59fe"

$description
[1] ""

$friendlyName
[1] "DanielSc"

$creationTime
[1] "2018-12-07T03:20:44.5994439+00:00"

$containerRegistry
[1] "/subscriptions/a947bb9f-f570-40a9-82cc-2fdd09f1553a/resourcegroups/amlworkshop/providers/microsoft.containerregistry/registries/danielscacrhfpssckt"

$keyVault
[1] "/subscriptions/a947bb9f-f570-40a9-82cc-2fdd09f1553a/resourcegroups/amlworkshop/providers/microsoft.keyvault/vaults/danielsckeyvaultvtydsccq"

$applicationInsights
[1] "/subscriptions/a947bb9f-f570-40a9-82cc-2fdd09f1553a/resourcegroups/amlworkshop/providers/microsoft.insights/components/danielscinsightsjewkxpor"

$identityPrincipalId
[1] "d9235e69-873f-494c-8c24-a45c2fdd9319"

$identityTenantId
[1] "72f988bf-86f1-41af-91ab-2d7cd011db47"

$identityType
[1] "SystemAssigned"

$storageAccount
[1] "/subscriptions/a947bb9f-f570-40a9-82cc-2fdd09f1553a/resourcegroups/amlworkshop/providers/microsoft.storage/storageaccounts/danielscstoragebuqznkny"